In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torch_cnn import Convolutional_neuralnet

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("CUDA is available. Using GPU.")
else:
  device = torch.device("cpu")
  print("CUDA is not available. Using CPU.")

use_gpu = True

In [ ]:
epochs = 10
batch_size = 64
num_output_classes = 10
learning_Rate = 0.001
use_gpu = True

In [ ]:
mnist_train = pd.read_csv('./data/mnist/mnist_train.csv', header=None)
mnist_test = pd.read_csv('./data/mnist/mnist_test.csv', header=None)

mnist_train_x = mnist_train.iloc[:, 1:].values / 255.0
mnist_train_y = mnist_train.iloc[:, 0].values 
mnist_test_x = mnist_test.iloc[:, 1:].values / 255.0
mnist_test_y = mnist_test.iloc[:, 0].values

if use_gpu:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32).to(device)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long).to(device)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32).to(device)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long).to(device)
else:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long)


train_dataset = TensorDataset(mnist_train_x, mnist_train_y)
test_dataset = TensorDataset(mnist_test_x, mnist_test_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

mnist_train_x.is_cuda


In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
  print(f"Training on {'GPU :)' if next(model.parameters()).is_cuda else 'CPU.. :('}\n")
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
            
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)
        
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
        
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)
        
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("=========================================")
    print()

In [ ]:
if use_gpu:
  model = Convolutional_neuralnet(train_loader).to(device)
else:
  model = Convolutional_neuralnet(train_loader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_Rate)


# cpu runtime: 436 seconds
# gpu runtime: 42.6 seconds
train_model(model, train_loader, test_loader, criterion, optimizer, 10)